In [7]:
import pandas as pd

# Define the path to your CSV file
#csv_file_path = 'data\loss_wheatwinter_countylevel_2000_2022.csv'
csv_file_path = r'data/usda_loss_wheatwinter_countylevel_2000_2022.csv'

# Load the CSV file into a dataframe
df = pd.read_csv(csv_file_path)
df.head(20)
len(df)

28776

In [ ]:
import ee
import pandas as pd
from ras_to_vec import *
import datetime

countyBoundary, winterWheatFC = ras_to_vec(df.iloc[28709]['State ANSI'], df.iloc[28709]['County'], df.iloc[28709]['Year'])

try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

# Define the year
year = int(df.iloc[28709]['Year'])

# Set the initial date to the first day of the year
start_date = datetime.datetime(year, 1, 1)

df_28709 = pd.DataFrame({'id': [2]})

# Iterate over each two-week period
while date.year == year:
    # Get the end date of the two-week period
    end_date = date + datetime.timedelta(weeks=2) - datetime.timedelta(days=1)

    # Load a Sentinel-2 image
    s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(countyBoundary) \
        .filterDate(date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 100))\
    
    # Get the first image in the collection
    s2m = ee.Image(s2.mosaic())

    # Calculate NDVI (Normalized Difference Vegetation Index)
    ndvi = s2m.normalizedDifference(['B8', 'B4'])

    # Clip the NDVI map with the winterWheatFC feature collection
    clippedNDVI = ndvi.clipToCollection(winterWheatFC)

    mean_ndvi = clippedNDVI.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=winterWheatFC,
        scale=30
    ).getInfo().get('nd')

    print(mean_ndvi)

    # For example, print the start and end dates
    print(date.strftime('%Y-%m-%d'))
    print(end_date.strftime('%Y-%m-%d'))

    df_28709[date.strftime('%Y-%m-%d')] = mean_ndvi

    # Move to the next two-week period
    date = end_date + datetime.timedelta(days=1)

In [1]:
import ee
import geemap
import pandas as pd
import folium

try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

state = str(20)
year = str(2014)
county = str('Thomas')

# Define the boundary of Thomas County, Kansas
countyBoundary = ee.FeatureCollection('TIGER/2018/Counties') \
    .filter(ee.Filter.eq('STATEFP', state)) \
    .filter(ee.Filter.eq('NAME', county)) 


start = year+'-01-01'
end = year+'-12-31'
# Load the CropScape data for the year 2022
cropscape = ee.ImageCollection('USDA/NASS/CDL') \
    .filter(ee.Filter.date(start, end)) \
    .first()

# Select winter wheat (crop code: 3) from the CropScape data
winterWheat = cropscape.select('cropland') \
    .eq(24) \
    .selfMask()

# Clip the winter wheat layer to Thomas County boundary
winterWheatClip = winterWheat.clip(countyBoundary)


# Convert the clipped layer to a FeatureCollection
winterWheatFC = winterWheatClip.reduceToVectors(
    geometry=countyBoundary,
    scale=30,
    geometryType='polygon',
    eightConnected=False,
    labelProperty='class'
)

In [20]:

# Create a map and add the clipped NDVI map
Map = geemap.Map(basemap = 'SATELLITE')
Map.centerObject(countyBoundary, 10)
palette = ['blue', 'white', 'green']
# Visualize the Landsat composite
visParams = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}  # Adjust the visualization parameters as needed
Map.addLayer(lands, visParams, 'Landsat Composite')
#Map.addLayer(clippedNDVI.select('nd'), {'min': -1, 'max': 1, 'palette': palette}, 'Clipped NDVI')
Map.addLayer(countyBoundary, {'color': 'FF000000'}, 'County Boundary')
#Map.addLayer(winterWheatFC, {'color': 'FF000000'}, 'Winter Wheat FC')
#Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Combined Image')

# Display the map
Map.addLayerControl()
Map

EEException: ImageCollection.reduce: Image 1_1_20220105T173719_20220105T173743_T14SKJ has a missing or invalid WRS_PATH: null.

In [2]:
import pandas as pd

# Define the path to your CSV file
#csv_file_path = 'data\loss_wheatwinter_countylevel_2000_2022.csv'
csv_file_path = r'data/usda_loss_wheatwinter_countylevel_2000_2022.csv'

# Load the CSV file into a dataframe
df = pd.read_csv(csv_file_path)

In [3]:
from get_ndvi_serie import get_ndvi_serie

get_ndvi_serie(df.iloc[26000])

AttributeError: 'Image' object has no attribute 'size'

In [8]:
from ras_to_vec import ras_to_vec
countyBoundary, winterWheatFC = ras_to_vec(20, 'Thomas', 2006)

In [9]:
countyBoundary.size().getInfo()

1

In [10]:
winterWheatFC.size().getInfo()

5378